In [1]:
import pandas as pd

In [2]:
def read_tbl():
    df = pd.read_excel(
        "../data/input/Invisible_Institue_Public_Records_Request_Response.xlsx"
    )

    df = df.rename(
        columns={
            "ACADEMYID": "person_nbr",
            "NAME": "name",
            "ACTIONEFFECTIVEDATE": "action_date",
            "ACTIONSTATUS": "action_status",
            "CERTIFICATION": "agency_name",
            "ISSUED": "certification_issue_date",
            "CURRENTCERTSTATUS": "current_certificatioan_status",
            "AGENCYORI": "agency_uid",
            "ACTIONTYPE": "action_type",
            "CERTIFICATIONTYPE": "certification_type",
        }
    )

    return df


def transform_employment_history(df):
    df["action_date"] = pd.to_datetime(df["action_date"])

    # Create a function to determine if a row represents a separation
    def is_separation(status):
        return "Separated" in str(status)

    employment_stints = []

    for (name, uid, cert), group in df.groupby(
        ["name", "person_nbr", "agency_name"]
    ):
        group = group.sort_values("action_date")

        # If there's only one row, handle it separately
        if len(group) == 1:
            row = group.iloc[0]
            stint_record = row.copy()
            stint_record["start_date"] = row["action_date"]
            # If it's a separation, use the same date as end_date
            stint_record["end_date"] = (
                row["action_date"]
                if is_separation(row["action_status"])
                else None
            )
            employment_stints.append(stint_record)
            continue

        separation_rows = group[group["action_status"].apply(is_separation)]

        if len(separation_rows) > 0:
            # Handle multiple separations for same agency
            for idx, separation_row in separation_rows.iterrows():
                # Find all rows before this separation
                mask = group["action_date"] <= separation_row["action_date"]
                stint_rows = group[mask]

                if len(stint_rows) > 0:  # Add check to ensure we have rows
                    # Get the earliest action_date as start_date
                    start_date = stint_rows["action_date"].min()
                    end_date = separation_row["action_date"]

                    # Create employment stint record using the most recent row's data
                    stint_record = stint_rows.iloc[-1].copy()
                    stint_record["start_date"] = start_date
                    stint_record["end_date"] = end_date
                    employment_stints.append(stint_record)

                # Remove processed rows for next iteration
                group = group[~mask]

        # Handle remaining rows (current employment with no separation)
        if len(group) > 0:
            stint_record = group.iloc[-1].copy()
            stint_record["start_date"] = group["action_date"].min()
            stint_record["end_date"] = None
            employment_stints.append(stint_record)

    if not employment_stints:
        return pd.DataFrame(
            columns=df.columns.tolist() + ["start_date", "end_date"]
        )

    result = pd.DataFrame(employment_stints)

    cols = ["start_date", "end_date"] + [
        col for col in result.columns if col not in ["start_date", "end_date"]
    ]
    result = result[cols]

    return result


def split_name(fullname):
    # Split on comma to separate last name from the rest
    parts = fullname.split(",")
    last_name = parts[0].strip()

    # Handle the first/middle/suffix part
    first_middle_suffix = parts[1].strip().split()

    # Get first name (first word after comma)
    first_name = first_middle_suffix[0]

    # Initialize middle name and suffix as empty strings
    middle_name = ""
    suffix = ""

    # If we have more than just the first name
    if len(first_middle_suffix) > 1:
        # Check if the last word is a common suffix
        common_suffixes = {"jr", "jr.", "sr", "sr.", "ii", "iii", "iv"}
        last_word = first_middle_suffix[-1].lower()

        if last_word in common_suffixes:
            # If there are words between first name and suffix, they form the middle name
            if len(first_middle_suffix) > 2:
                middle_name = " ".join(first_middle_suffix[1:-1])
            suffix = first_middle_suffix[-1]
        else:
            # If no suffix, all words after first name form the middle name
            middle_name = " ".join(first_middle_suffix[1:])

    return first_name, middle_name, last_name, suffix


def fix_dates(df):
    df.loc[:, "action_date"] = pd.to_datetime(df.action_date, errors="coerce")
    return df


def split_names(df):
    df[["first_name", "middle_name", "last_name", "suffix"]] = df["name"].apply(
        lambda x: pd.Series(split_name(x))
    )
    return df


def normalize_cols(df):
    df.loc[:, "end_date"] = df.end_date.astype(str).str.replace(
        r"NaT", "", regex=True
    )
    df.loc[:, "first_name"] = df.first_name.str.lower()
    df.loc[:, "middle_name"] = df.middle_name.str.lower()
    df.loc[:, "last_name"] = df.last_name.str.lower()
    df.loc[:, "suffix"] = df.suffix.str.lower()
    return df


def extract_rank(df):
    df.loc[:, "rank"] = (
        df.agency_name.str.lower()
        .str.strip()
        .str.replace(r"(.+) - (\w+)? ?(\w+) (\w+)$", r"\2 \3 \4", regex=True)
        .str.replace(r"deput y", "deputy", regex=False)
        .str.replace(r"law enforcement ", "", regex=False)
    )
    return df


def clean_agency_name(df):
    df.loc[:, "agency_name"] = (
        df.agency_name.str.lower()
        .str.strip()
        .str.replace(r" - (\w+)? ?(\w+) (\w+)$", "", regex=True)
        .str.replace(r"pd$", "police department", regex=True)
        .str.replace(r"\bco\.", "county", regex=True)
        .str.replace(r"&", "and", regex=False)
        .str.replace(r"law enforcement officer - ", "", regex=False)
    )
    return df


def filter_cols(df):
    df = df[
        [
            "start_date",
            "end_date",
            "person_nbr",
            "agency_name",
            "rank",
            "first_name",
            "middle_name",
            "last_name",
            "suffix",
        ]
    ]
    return df


def drop_rows_w_no_start_date(df):
    df = df[~((df.start_date.fillna("") == ""))]
    return df


df = read_tbl()


df = (
    df.pipe(fix_dates)
    .pipe(transform_employment_history)
    .pipe(split_names)
    .pipe(normalize_cols)
    .pipe(extract_rank)
    .pipe(clean_agency_name)
    .pipe(filter_cols)
    .pipe(drop_rows_w_no_start_date)
)

# actiontype: issue import references a start_date, actiontype update[import]  and action_status separated indicates an end_date

df

,start_date,end_date,person_nbr,agency_name,rank,first_name,middle_name,last_name,suffix
2060,1999-12-01,,0012-1559,asheville police department,officer,sean,thomas,aardema,
67088,1979-11-02,2005-05-01,0012-1548,asheville police department,officer,thomas,frederick,aardema,
44915,2001-04-12,,0012-1548,buncombe county sheriff's office,deputy sheriff,thomas,frederick,aardema,
67091,2011-05-17,2012-06-28,0012-1548,marion police department,officer,thomas,frederick,aardema,
25934,2019-12-19,,2944-5526,butner public safety,officer,justin,wayne,aaron,
...,...,...,...,...,...,...,...,...,...
48034,1998-12-07,,0700-8529,burke county sheriff's office,deputy sheriff,david,c,warlick,
124994,1998-07-31,2004-05-17,0700-8529,long view police department,officer,david,c,warlick,
124996,2005-05-17,2022-09-21,0700-8529,nc dept of revenue-unauthorized substance tax ...,officer,david,c,warlick,
1174,2022-09-06,,6513-0135,pamlico county sheriff's office,deputy sheriff,brian,,wielhouwer,


In [3]:
df.agency_name.unique()

array(['asheville police department', "buncombe county sheriff's office",
       'marion police department', 'butner public safety',
       'lincolnton police department', "haywood county sheriff's office",
       'hamlet police department', "scotland county sheriff's office",
       'wagram police department', "davidson county sheriff's office",
       "harnett county sheriff's office", 'zebulon police department',
       'company police comm - raleigh', 'wake forest police department',
       "surry county sheriff's office",
       'charlotte-mecklenburg police department',
       "wake county sheriff's office", 'raleigh police department',
       'beech mountain police department', 'greensboro police department',
       "new hanover county sheriff's office",
       'tabor city police department', "catawba county sheriff's office",
       "chatham county sheriff's office", 'reidsville police department',
       'erwin police department', 'hope mills police department',
       'lake w

In [4]:
df.sort_values("person_nbr")

,start_date,end_date,person_nbr,agency_name,rank,first_name,middle_name,last_name,suffix
3492,2009-06-29,,****0019-8088,kannapolis police department,officer,david,mark,zienka,ii
39620,2009-11-25,,*0097-2608,wake county sheriff's office,deputy sheriff,charles,eugene,lambert,
81031,1983-05-19,1983-08-31,*0097-2608,wake county sheriff's office,officer,charles,eugene,lambert,
7181,2019-02-19,,*0097-2608,nc state capitol police,officer,charles,eugene,lambert,
81027,1999-09-14,2001-09-15,*0097-2608,clayton police department,officer,charles,eugene,lambert,
...,...,...,...,...,...,...,...,...,...
200322,1981-07-23,1988-12-14,9998-8200,franklinton police department,officer,jimmy,winston,oneal,
248084,1998-11-23,2000-05-12,9998-8909,wilson county sheriff's office,deputy sheriff,chad,holmes,fowler,
202596,1998-07-09,1998-10-13,9998-8909,richlands police department,officer,chad,holmes,fowler,
203005,1987-06-10,1987-11-12,9998-9251,woodfin police department,officer,alan,kenneth,wren,


In [5]:
df.sort_values("person_nbr").head(50)

,start_date,end_date,person_nbr,agency_name,rank,first_name,middle_name,last_name,suffix
3492,2009-06-29,,****0019-8088,kannapolis police department,officer,david,mark,zienka,ii
39620,2009-11-25,,*0097-2608,wake county sheriff's office,deputy sheriff,charles,eugene,lambert,
81031,1983-05-19,1983-08-31,*0097-2608,wake county sheriff's office,officer,charles,eugene,lambert,
7181,2019-02-19,,*0097-2608,nc state capitol police,officer,charles,eugene,lambert,
81027,1999-09-14,2001-09-15,*0097-2608,clayton police department,officer,charles,eugene,lambert,
81029,1999-12-14,2008-12-01,*0097-2608,knightdale police department,officer,charles,eugene,lambert,
39621,1983-09-01,1998-09-16,*0097-2608,wake county sheriff's office,deputy sheriff,charles,eugene,lambert,
80492,2003-01-03,2010-05-31,*3087-0896,north carolina state highway patrol,officer,william,irwin,gray,
80483,1997-06-12,1998-05-25,*3087-0896,asheville police department,officer,william,irwin,gray,
80489,1998-09-25,2002-12-31,*3087-0896,nc dmv - license and theft,officer,william,irwin,gray,


In [6]:
## done

df.to_csv("../data/output/north-carolina-processed.csv", index=False)